<a href="https://colab.research.google.com/github/GabsLUZ/Trabalho_Final_IA/blob/main/Melhorando_Resultados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-plot

In [ ]:
# Manipulacao de dados
import numpy as np
import pandas as pd

# Visualizacao de dados
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt #teve que instalar

# Machine learning 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# balanceamento
from imblearn.under_sampling import RandomUnderSampler

# Grid e Random Search
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

# cross validate
from sklearn.model_selection import StratifiedKFold, cross_validate, KFold 
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, precision_recall_curve #mudou

# Selecao de caracteristicas e encoders
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

# Metricas de avaliacao
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, average_precision_score,RocCurveDisplay # versão diferente

# remover warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from itertools import product
from  sklearn.utils.multiclass  import  unique_labels
from sklearn.utils import check_matplotlib_support

class ConfusionMatrixDisplay:
    def __init__(self, confusion_matrix, *, display_labels=None):
        self.confusion_matrix = confusion_matrix
        self.display_labels = display_labels

    def plot(self, *, include_values=True, cmap='Blues',
             xticks_rotation='horizontal', values_format=None,
             ax=None, colorbar=True):
        
        import matplotlib.pyplot as plt

        if ax is None:
            fig, ax = plt.subplots()
        else:
            fig = ax.figure

        cm = self.confusion_matrix
        n_classes = cm.shape[0]
        self.im_ = ax.imshow(cm, interpolation='nearest', cmap=cmap)
        self.text_ = None
        cmap_min, cmap_max = self.im_.cmap(0), self.im_.cmap(1.0)

        if include_values:
            self.text_ = np.empty_like(cm, dtype=object)

            # print text with appropriate color depending on background
            thresh = (cm.max() + cm.min()) / 2.0

            for i, j in product(range(n_classes), range(n_classes)):
                color = cmap_max if cm[i, j] < thresh else cmap_min

                if values_format is None:
                    text_cm = format(cm[i, j], '.2g')
                    if cm.dtype.kind != 'f':
                        text_d = format(cm[i, j], 'd')
                        if len(text_d) < len(text_cm):
                            text_cm = text_d
                else:
                    text_cm = format(cm[i, j], values_format)

                self.text_[i, j] = ax.text(
                    j, i, text_cm,
                    ha="center", va="center",
                    color=color)

        if self.display_labels is None:
            display_labels = np.arange(n_classes)
        else:
            display_labels = self.display_labels
        if colorbar:
            fig.colorbar(self.im_, ax=ax)
        ax.set(xticks=np.arange(n_classes),
               yticks=np.arange(n_classes),
               xticklabels=display_labels,
               yticklabels=display_labels,
               ylabel="Valor Real",
               xlabel="Valor Predito")

        ax.set_ylim((n_classes - 0.5, -0.5))
        plt.setp(ax.get_xticklabels(), rotation=xticks_rotation)

        self.figure_ = fig
        self.ax_ = ax
        return self

    @classmethod
    def from_estimator(
        cls,
        estimator,
        X,
        y,
        *,
        labels=None,
        sample_weight=None,
        normalize=None,
        display_labels=None,
        include_values=True,
        xticks_rotation="horizontal",
        values_format=None,
        cmap="Blues",
        ax=None,
        colorbar=True,
    ):
        
        method_name = f"{cls.__name__}.from_estimator"
        
        if not is_classifier(estimator):
            raise ValueError(f"{method_name} only supports classifiers")
        y_pred = estimator.predict(X)

        return cls.from_predictions(
            y,
            y_pred,
            sample_weight=sample_weight,
            labels=labels,
            normalize=normalize,
            display_labels=display_labels,
            include_values=include_values,
            cmap=cmap,
            ax=ax,
            xticks_rotation=xticks_rotation,
            values_format=values_format,
            colorbar=colorbar,
        )

    @classmethod
    def from_predictions(
        cls,
        y_true,
        y_pred,
        *,
        labels=None,
        sample_weight=None,
        normalize=None,
        display_labels=None,
        include_values=True,
        xticks_rotation="horizontal",
        values_format=None,
        cmap="Blues",
        ax=None,
        colorbar=True,
    ):

        check_matplotlib_support(f"{cls.__name__}.from_predictions")

        if display_labels is None:
            if labels is None:
                display_labels = unique_labels(y_true, y_pred)
            else:
                display_labels = labels

        cm = confusion_matrix(
            y_true,
            y_pred,
            sample_weight=sample_weight,
            labels=labels,
            normalize=normalize,
        )

        disp = cls(confusion_matrix=cm, display_labels=display_labels)

        return disp.plot(
            include_values=include_values,
            cmap=cmap,
            ax=ax,
            xticks_rotation=xticks_rotation,
            values_format=values_format,
            colorbar=colorbar,
        )


def plot_confusion_matrix_edit(estimator, X, y_true, *, labels=None,
                          sample_weight=None, normalize=None,
                          display_labels=None, include_values=True,
                          xticks_rotation='horizontal',
                          values_format=None,
                          cmap='Blues', ax=None, colorbar=True):

    y_pred = estimator.predict(X)
    cm = confusion_matrix(y_true, y_pred, sample_weight=sample_weight,
                          labels=labels, normalize=normalize)

    if display_labels is None:
        if labels is None:
            display_labels = unique_labels(y_true, y_pred)
        else:
            display_labels = labels

    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=display_labels)
    return disp.plot(include_values=include_values,
                     cmap=cmap, ax=ax, xticks_rotation=xticks_rotation,
                     values_format=values_format, colorbar=colorbar)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/trabalho IA/covid_2022_prepared.csv", sep=';')

In [ ]:
def get_ratios(y):
    """
        y: y_train ou y_test
        retorna total e porcentagens
    """
    
    df1 = y.value_counts().rename_axis('OBITO').reset_index(name='Total')
    df2 = (y.value_counts(normalize=True)*100).rename_axis('OBITO').reset_index(name='Porcentagem').drop(columns='OBITO')
    df_concat = pd.concat([df1, df2], axis=1)

    return df_concat
    

In [ ]:
X_ = df.drop(['EVOLUCAO'], axis=1)
y_ = df['EVOLUCAO']

In [ ]:
# Validacao
X, X_test_, y, y_test_ = train_test_split(X_, y_, stratify=y_, test_size=0.3, random_state=777)

In [ ]:
y.reset_index(drop=True, inplace=True)

In [ ]:
X.reset_index(drop=True, inplace=True)

In [ ]:
get_ratios(y)

In [ ]:
get_ratios(y_test_)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve, roc_curve, auc


# balanceamento
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline 
from statistics import mean, stdev
from collections import Counter

def print_metric(results, name):
    media = np.mean(results)
    desvio_padrao = np.std(results)
    print(f"{name} ", end='')
    print("médio: %.3f | intervalo: [%.3f, %.3f]" % ((media), (media - 2 * desvio_padrao), (media + 2 * desvio_padrao)))

In [ ]:
# initial parameters
SEED = 777
K_FOLD = 5

models = [RandomForestClassifier(random_state=SEED)]
verbose_models = ['Random Forest Classifier']


#undersampler
rus = RandomUnderSampler(random_state=SEED)

plt.rcParams.update({'font.size': 20})

for name, model in zip(verbose_models, models):
    print("===============")
    print(name + " - ORIGINAL")
    
    auc_roc = []
    precision = []
    recall = []
    f1score = []
    aucscore = []
    # kfold
    kf = StratifiedKFold(n_splits=K_FOLD, shuffle=True, random_state=SEED)

    # percorre cada fold      
    for train_index, test_index in kf.split(X, y):
        # divisao treino e teste
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # treino e predicao
        y_pred = model.fit(X_train, y_train).predict(X_test)

        # computacao da metrica
        auc_roc.append(roc_auc_score(y_test, y_pred))
        fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=2)
        aucscore.append(auc(fpr, tpr))
        precision.append(precision_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        f1score.append(f1_score(y_test, y_pred))
        

    print_metric(auc_roc, 'AUC-ROC')
    print_metric(precision, 'PRECISÃO')
    print_metric(recall, 'REVOCAÇÃO')
    print_metric(f1score, 'F1 SCORE')
    print_metric(aucscore, 'AUC')
    print(confusion_matrix(y_test, y_pred))

    # MATRIZ DE CONFUSAO
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    #fig, ax2 = plt.subplots(figsize=(10, 10))
    
    #plot_confusion_matrix_edit(model, X_test_, y_test_, values_format= '.2%', normalize='true', 
      #                display_labels=['Recuperado', 'Óbito'], ax=ax1, colorbar=False); 

    print("===============")

    auc_roc = []
    precision = []
    recall = []
    f1score = []
    aucscore = []

    # kfold
    kf = StratifiedKFold(n_splits=K_FOLD, shuffle=True, random_state=SEED)

    print(name + " - BALANCEADO")
    
    # pipeline do modelo
    pipeline = make_pipeline(rus, model)
    
    for train_index, test_index in kf.split(X,y):
        # divisao treino e teste
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # treino e predicao
        y_pred = pipeline.fit(X_train, y_train).predict(X_test)

        # computacao da metrica
        auc_roc.append(roc_auc_score(y_test, y_pred))
        fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=2)
        aucscore.append(auc(fpr, tpr))
        
        precision.append(precision_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        f1score.append(f1_score(y_test, y_pred))

    print_metric(auc_roc, 'AUC-ROC')
    print_metric(precision, 'PRECISÃO')
    print_metric(recall, 'REVOCAÇÃO')
    print_metric(f1score, 'F1 SCORE')
    print_metric(aucscore, 'AUC')
    print(confusion_matrix(y_test, y_pred))  

    plot_confusion_matrix_edit(model, X_test_, y_test_,
                      display_labels=['Recuperado', 'Óbito'], ax=ax1, colorbar=True);

    plot_confusion_matrix_edit(model, X_test_, y_test_, values_format= '.2%', normalize='true',
                      display_labels=['Recuperado', 'Óbito'], ax=ax2, colorbar=True); 

    #ax2.title.set_text(f"Matriz de confusão - {name} \nSubamostragem")
    plt.tight_layout()
    plt.show() 

    print("===============")